## Prácticas recomendadas para administrar los datos de serie temporal en DynamoDB

Los principios generales de diseño de Amazon DynamoDB recomiendan utilizar la menor cantidad de tablas posible. En la mayoría de las aplicaciones, solo se necesita una tabla. Sin embargo, para los datos de series temporales, a menudo lo mejor para administrarlos es usar una tabla por aplicación y periodo.

### Patrón de diseño de los datos de serie temporal
Imagine un caso típico de una serie temporal en el que quiere hacer un seguimiento de una gran cantidad de eventos. Tiene un patrón de acceso de escritura que establece que se registren todos los eventos con la fecha de hoy. El patrón de acceso de lectura podría establecer que los eventos de hoy se lean con más frecuencia, que los eventos de ayer se lean con mucha menos frecuencia y que los eventos más antiguos apenas se lean. Una manera de administrarlo consiste en incorporar la fecha y hora actuales en la clave principal.

Normalmente, el siguiente patrón de diseño sirve para administrar este tipo de escenarios eficazmente:

* Cree una tabla por periodo, aprovisionada con la capacidad requerida de lectura y escritura y con los índices que se necesitan.

* Antes de que termine cada periodo, precompile la tabla para el siguiente periodo. Justo cuando termine el período actual, dirija el tráfico de los eventos a la nueva tabla. Puede asignar nombres a estas tablas que indiquen los períodos que contienen.

* Tan pronto como la tabla deje de estar disponible para escribir en ella, reduzca su capacidad de escritura aprovisionada a un valor menor (por ejemplo, 1 WCU) y aprovisione la capacidad de lectura apropiada, según proceda. Reduzca la capacidad de lectura aprovisionada de las tablas anteriores a medida que vayan venciendo. Puede optar por archivar o eliminar las tablas cuyo contenido va a necesitar en pocas ocasiones o no va a necesitar nunca.

Se trata de asignar, para el periodo actual, los recursos requeridos que vayan a experimentar el máximo volumen de tráfico y de reducir el aprovisionamiento de las tablas más antiguas que no se utilizan activamente, con lo que se ahorra en costos. En función de sus necesidades de negocio, podría plantearse fragmentar la escritura con el fin de distribuir el tráfico de manera uniforme en la clave de partición lógica.


In [1]:
import boto3
import random
import uuid
from botocore.exceptions import ClientError
import pandas as pd
from spdynamodb import DynamoTable
import json
import time
from decimal import Decimal
from datetime import datetime, timedelta

### Crear 3 tablas para los datos de serie temporal

In [2]:
today = datetime.today().strftime('%Y-%m-%d')
yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
before_yesterday = (datetime.today() - timedelta(days=2)).strftime('%Y-%m-%d')

table_name_today = 'TimeSeries-' + today
table_name_yesterday = 'TimeSeries-' + yesterday
table_name_before_yesterday = 'TimeSeries-' + before_yesterday

print("Table name today:            ", table_name_today)
print("Table name yesterday:        ", table_name_yesterday)
print("Table name before yesterday: ", table_name_before_yesterday)

Table name today:             TimeSeries-2023-06-23
Table name yesterday:         TimeSeries-2023-06-22
Table name before yesterday:  TimeSeries-2023-06-21


In [3]:
dt_now=DynamoTable()
try:
    dt_now.select_table(table_name_today)
    print(dt)
except:
    dt_now.create_table(
        table_name=table_name_today,
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S',
        wcu=300,
        rcu=100
    )

Table created successfully!


In [4]:
dt_yesterday=DynamoTable()
try:
    dt_yesterday.select_table(table_name_yesterday)
    print(dt)
except:
    dt_yesterday.create_table(
        table_name=table_name_yesterday,
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S',
        wcu=1,
        rcu=50
    )

Table created successfully!


In [5]:
dt_byesterday=DynamoTable()
try:
    dt_byesterday.select_table(table_name_before_yesterday)
    print(dt)
except:
    dt_byesterday.create_table(
        table_name=table_name_before_yesterday,
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S',
        wcu=1,
        rcu=1
    )

Table created successfully!


In [6]:
def random_time():
    hour = random.randint(0, 23)
    minute = random.randint(0, 59)
    second = random.randint(0, 59)
    mills = random.randint(2, 500)
    return f'{hour:02}:{minute:02}:{second:02}.{mills:03}'

def radian_int():
    return str(random.randint(15000, 28000)) + ' W/Sr'

def wavelength_int():
    return str(random.randint(400, 700)) + ' nm'

def frequency_int():
    return str(random.randint(400, 700)) + ' THz'

def amplitude_int():
    return str(random.randint(0, 100)) + ' dBm'

#?==================== Generate Data ====================
def generate_data(total_items=300, day="Today"):
    count_total = 0
    if day == "Today":
        day_date = datetime.today().strftime('%Y-%m-%d')
    elif day == "Yesterday":
        day_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
    elif day == "Before Yesterday":
        day_date = (datetime.today() - timedelta(days=2)).strftime('%Y-%m-%d')
    else:
        print("Error day must be: 'Today', 'Yesterday' or 'Before Yesterday'")
        return
    pk, sk, RadiantIntensity, Wavelenght, Frequency, Amplitude = [], [], [], [], [], []
    for i in range(total_items):
        if count_total == total_items:
            break
        else:
            pk.append(day_date)
            sk.append(random_time())
            RadiantIntensity.append(radian_int())
            Wavelenght.append(wavelength_int())
            Frequency.append(frequency_int())
            Amplitude.append(amplitude_int())
        
    df_main = pd.DataFrame(
        {
            "PK": pk,
            "SK": sk,
            "RadiantIntensity": RadiantIntensity,
            "Wavelenght": Wavelenght,
            "Frequency": Frequency,
            "Amplitude": Amplitude
        }
    )   
    return df_main


In [7]:
df = generate_data(total_items=300, day="Today")
dt_now.batch_pandas(df)
df = generate_data(total_items=150, day="Yesterday")
dt_yesterday.batch_pandas(df)
df = generate_data(total_items=50, day="Before Yesterday")
dt_byesterday.batch_pandas(df)

### Cambio de periodo de la tabla

In [8]:
# Create table for tomorrow
tomorrow = datetime.now() + timedelta(days=1)
table_name_tomorrow = 'TimeSeries-' + str(tomorrow.strftime('%Y-%m-%d'))
print("Table name tomorrow: ", table_name_tomorrow)

Table name tomorrow:  TimeSeries-2023-06-24


In [9]:
dt_tomorrow=DynamoTable()
try:
    dt_tomorrow.select_table(table_name_tomorrow)
    print(dt)
except:
    dt_tomorrow.create_table(
        table_name=table_name_tomorrow,
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S',
        wcu=300,
        rcu=100
    )

Table created successfully!


In [10]:
# Change ProvisionedThroughput
dt_now.table.update(
    ProvisionedThroughput={
        'ReadCapacityUnits': 50,
        'WriteCapacityUnits': 1
    }
)

dynamodb.Table(name='TimeSeries-2023-06-23')

In [11]:
# Delete table before yesterday
dt_byesterday.table.delete()

{'TableDescription': {'TableName': 'TimeSeries-2023-06-21',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 1,
   'WriteCapacityUnits': 1},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:234736309150:table/TimeSeries-2023-06-21',
  'TableId': 'b78c9284-78fc-454d-ae04-3393262f70e0',
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': 'ON2E0JQ6OAL0R11J7JU7IRQHQFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 23 Jun 2023 14:09:48 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '378',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ON2E0JQ6OAL0R11J7JU7IRQHQFVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '62922297'},
  'RetryAttempts': 0}}

In [12]:
# Change ProvisionedThroughput from table yesterday
dt_yesterday.table.update(
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    } 
)

dynamodb.Table(name='TimeSeries-2023-06-22')